In [1]:
import sys
from enum import Enum, auto
sys.version

'3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]'

In [2]:
from enum import Enum, auto

class Color(Enum):
    RED: int = 1
    GREEN: int = 2
    BLUE: int = 3

In [3]:
class Color(Enum):
    RED: int = auto()
    GREEN: int = auto()
    BLUE: int = auto()

In [4]:
c1 = Color(1)
c2 = Color(Color.RED)
c3 = Color(Color['RED'])
print(type(Color), type(c1), type(Color.RED), type(Color['RED']))

<class 'enum.EnumMeta'> <enum 'Color'> <enum 'Color'> <enum 'Color'>


In [5]:
Color(1) == 1             # false
Color.RED == Color(1)     # true
Color.RED == Color['RED'] # true

True

In [6]:
print(len(Color)) # 3
# can iterate:
for c in Color:
    print(c)

3
Color.RED
Color.GREEN
Color.BLUE


In [7]:
# make auto() cleaner
# add __str__() in EnumMeta

class auto():
    pass

class EnumMeta(type):
    def __new__(metacls, cls, bases, classdict, **kwds):
        enumerations = {x: y for x, y in classdict.items() if not x.startswith('__')}
        # handle auto() and auto
        next_value = 1
        for k, v in enumerations.items():
            if type(v) is auto or v is auto:
                enumerations[k] = next_value
                next_value += 1
            else:
                next_value = v + 1
        enum = super().__new__(metacls, cls, bases, classdict, **kwds)
        enum._enumerations = enumerations
        return enum

    def __str__(cls):
        return f'<enum \'{cls.__name__}\'>'
    
    def __len__(cls):
        return len(cls._enumerations)
    
    def __iter__(cls):
        return (cls(value) for value in cls._enumerations.values())
        
class Enum(metaclass=EnumMeta):
    def __init__(self, value):
        # make sure the passed in value is a valid enumeration value
        if value not in self.__class__._enumerations.values():
            raise ValueError(f'{value} is not a valid {self.__class__.__name__}')
        # save the actual enumeration value
        for k, v in self.__class__._enumerations.items():
            if v == value:
                self.__key = k
                self.__value = v
    
    def __str__(self):
        return "%s.%s" % (self.__class__.__name__, self.__key)
    
class Color(Enum):
    RED: int = auto
    GREEN: int = auto
    BLUE: int = auto

print(Color)           # __str__() in EnumMeta
print(type(Color(1)))  # __str__() in EnumMeta
print(Color(1))        # __str__() in Enum

<enum 'Color'>
<enum 'Color'>
Color.RED


In [8]:
# make Color['RED'] return a Color.RED using __getitem__() in EnumMeta

class auto():
    pass

class EnumMeta(type):
    def __new__(metacls, cls, bases, classdict, **kwds):
        enumerations = {x: y for x, y in classdict.items() if not x.startswith('__')}
        # handle auto() and auto
        next_value = 1
        for k, v in enumerations.items():
            if type(v) is auto or v is auto:
                enumerations[k] = next_value
                next_value += 1
            else:
                next_value = v + 1
        enum = super().__new__(metacls, cls, bases, classdict, **kwds)
        enum._enumerations = enumerations
        return enum

    def __str__(cls):
        return f'<enum \'{cls.__name__}\'>'
    
    def __len__(cls):
        return len(cls._enumerations)
    
    def __iter__(cls):
        return (cls(value) for value in cls._enumerations.values())
    
    def __getitem__(cls, key):
        return cls(cls._enumerations[key])
        
class Enum(metaclass=EnumMeta):
    def __init__(self, value):
        # make sure the passed in value is a valid enumeration value
        if value not in self.__class__._enumerations.values():
            raise ValueError(f'{value} is not a valid {self.__class__.__name__}')
        # save the actual enumeration value
        for k, v in self.__class__._enumerations.items():
            if v == value:
                self.__key = k
                self.__value = v
    
    def __str__(self):
        return "%s.%s" % (self.__class__.__name__, self.__key)
    
class Color(Enum):
    RED: int = auto
    GREEN: int = auto()
    BLUE: int = auto()

print(Color['RED'])      # returns a Color.RED object
 #print(Color['PURPLE']) # KeyError: 'PURPLE'

Color.RED


In [9]:
# make Color.RED return a Color.RED object, instead of 1

class auto():
    pass

class EnumMeta(type):
    def __new__(metacls, cls, bases, classdict, **kwds):
        enumerations = {x: y for x, y in classdict.items() if not x.startswith('__')}
        # handle auto() and auto
        next_value = 1
        for k, v in enumerations.items():
            if type(v) is auto or v is auto:
                enumerations[k] = next_value
                next_value += 1
            else:
                next_value = v + 1
        enum = super().__new__(metacls, cls, bases, classdict, **kwds)
        enum._enumerations = enumerations
        return enum

    def __str__(cls):
        return f'<enum \'{cls.__name__}\'>'
    
    def __len__(cls):
        return len(cls._enumerations)
    
    def __iter__(cls):
        return (cls(value) for value in cls._enumerations.values())
        
    def __getitem__(cls, key):
        return cls(cls._enumerations[key])
    
    def __getattribute__(cls, key):
        # return cls(cls._enumerations[key]) # recursively call itself, kills the kernel!
        if key.startswith('_'):
            return object.__getattribute__(cls, key)
        else:
            return cls(object.__getattribute__(cls, '_enumerations')[key])
        
class Enum(metaclass=EnumMeta):
    def __init__(self, value):
        # make sure the passed in value is a valid enumeration value
        if value not in self.__class__._enumerations.values():
            raise ValueError(f'{value} is not a valid {self.__class__.__name__}')
        # save the actual enumeration value
        for k, v in self.__class__._enumerations.items():
            if v == value:
                self.__key = k
                self.__value = v
    
    def __str__(self):
        return "%s.%s" % (self.__class__.__name__, self.__key)
    
class Color(Enum):
    RED: int = auto()
    GREEN: int = auto()
    BLUE: int = auto()

print(Color['RED']) # returns a Color.RED object
print(Color.RED)    # returns a Color.RED object
print(Color(1))
print(len(Color))
for c in Color: print(c)
# print(Color.PURPLE) # KeyError: 'PURPLE'

Color.RED
Color.RED
Color.RED
3
Color.RED
Color.GREEN
Color.BLUE


In [10]:
# add __contains__

class auto():
    pass

class EnumMeta(type):
    def __new__(metacls, cls, bases, classdict, **kwds):
        enumerations = {x: y for x, y in classdict.items() if not x.startswith('__')}
        # handle auto() and auto
        next_value = 1
        for k, v in enumerations.items():
            if type(v) is auto or v is auto:
                enumerations[k] = next_value
                next_value += 1
            else:
                next_value = v + 1
        enum = super().__new__(metacls, cls, bases, classdict, **kwds)
        enum._enumerations = enumerations
        return enum

    def __str__(cls):
        return f'<enum \'{cls.__name__}\'>'
    
    def __len__(cls):
        return len(cls._enumerations)
    
    def __iter__(cls):
        return (cls(value) for value in cls._enumerations.values())
        
    def __getitem__(cls, key):
        return cls(cls._enumerations[key])
    
    def __getattribute__(cls, key):
        if key.startswith('_'):
            return object.__getattribute__(cls, key)
        else:
            return cls(object.__getattribute__(cls, '_enumerations')[key])

    def __contains__(cls, other):
        if type(other) == cls:
            return True
        else:
            return False        

class Enum(metaclass=EnumMeta):
    def __init__(self, value):
        # make sure the passed in value is a valid enumeration value
        if value not in self.__class__._enumerations.values():
            raise ValueError(f'{value} is not a valid {self.__class__.__name__}')
        # save the actual enumeration value
        for k, v in self.__class__._enumerations.items():
            if v == value:
                self.__key = k
                self.__value = v
    
    def __str__(self):
        return "%s.%s" % (self.__class__.__name__, self.__key)
    
class Color(Enum):
    RED: int = auto()
    GREEN: int = auto()
    BLUE: int = auto()

class WeekendDay(Enum):
    SATURDAY: int = auto()
    SUNDAY: int = auto()        

print(Color['RED'] in Color)
print(Color.RED in Color)
print(Color(1) in Color)
print(Color.RED in WeekendDay)
print(Color.BLUE in WeekendDay)

True
True
True
False
False


In [16]:
# fix truthiness with __eq__

class auto():
    pass

class EnumMeta(type):
    def __new__(metacls, cls, bases, classdict, **kwds):
        enumerations = {x: y for x, y in classdict.items() if not x.startswith('__')}
        # handle auto() and auto
        next_value = 1
        for k, v in enumerations.items():
            if type(v) is auto or v is auto:
                enumerations[k] = next_value
                next_value += 1
            else:
                next_value = v + 1
        enum = super().__new__(metacls, cls, bases, classdict, **kwds)
        enum._enumerations = enumerations
        return enum

    def __str__(cls):
        return f'<enum \'{cls.__name__}\'>'
    
    def __len__(cls):
        return len(cls._enumerations)
    
    def __iter__(cls):
        return (cls(value) for value in cls._enumerations.values())
        
    def __getitem__(cls, key):
        return cls(cls._enumerations[key])
    
    def __getattribute__(cls, key):
        if key.startswith('_'):
            return object.__getattribute__(cls, key)
        else:
            return cls(object.__getattribute__(cls, '_enumerations')[key])

    def __contains__(cls, other):
        if type(other) == cls:
            return True
        else:
            return False
        
class Enum(metaclass=EnumMeta):
    def __init__(self, value):
        # make sure the passed in value is a valid enumeration value
        if value not in self.__class__._enumerations.values():
            raise ValueError(f'{value} is not a valid {self.__class__.__name__}')
        # save the actual enumeration value
        for k, v in self.__class__._enumerations.items():
            if v == value:
                self.__key = k
                self.__value = v
    
    def __str__(self):
        return "%s.%s" % (self.__class__.__name__, self.__key)
    
    def __eq__(self, other):
        if type(self) != type(other):
            return False
        else:
            return (self.__key == other.__key and self.__value == other.__value)
    
class Color(Enum):
    RED: int = auto()
    GREEN: int = auto()
    BLUE: int = auto()

class WeekendDay(Enum):
    SATURDAY: int = auto()
    SUNDAY: int = auto()
        
print(Color.RED == Color)               # False
print(Color.RED == Color.RED)           # True
print(Color.RED is Color.RED)           # True
print(Color.RED == Color['RED'])        # True
print(Color.RED == Color(1))            # True
print(Color.RED == Color(2))            # False
print(Color.RED == WeekendDay.SATURDAY) # False

False
True
False
True
True
False
False


In [18]:
print(Color.RED is Color.RED)
print(Color.RED is Color['RED'])
print(Color.RED is Color(1))

False
False
False


In [19]:
# make static objects and keep returning those

class auto():
    pass

class EnumMeta(type):
    def __new__(metacls, cls, bases, classdict, **kwds):
        print(f'Constructing new type {cls}')
        enumerations = {x: y for x, y in classdict.items() if not x.startswith('__')}
        # handle auto() and auto
        next_value = 1
        for k, v in enumerations.items():
            if type(v) is auto or v is auto:
                enumerations[k] = next_value
                next_value += 1
            else:
                next_value = v + 1
        enum = super().__new__(metacls, cls, bases, classdict, **kwds)
        enum._enumerations = enumerations
        reverse_enumerations = {y: x for x, y in enumerations.items()}
        # make "static" instances of each enumeration object
        enum._instances = {k: object.__new__(enum) for k, v in enumerations.items()}
        # initialize static instances
        for k, instance in enum._instances.items():
            instance.__init__(enumerations[k])
        # overwrite the new Enum's __new__() so that is returns the static instances
        enum.__new__ = lambda cls, value: enum._instances[reverse_enumerations[value]]
        return enum

    def __str__(cls):
        return f'<enum \'{cls.__name__}\'>'
    
    def __len__(cls):
        return len(cls._enumerations)
    
    def __iter__(cls):
        return (cls(value) for value in cls._enumerations.values())
        
    def __getitem__(cls, key):
        return cls(cls._enumerations[key])
    
    def __getattribute__(cls, key):
        if key.startswith('_'):
            return object.__getattribute__(cls, key)
        else:
            return cls(object.__getattribute__(cls, '_enumerations')[key])
        
    def __contains__(cls, other):
        if type(other) == cls:
            return True
        else:
            return False
        
class Enum(metaclass=EnumMeta):    
    def __init__(self, value):
        if hasattr(self, '_Enum__key') and hasattr(self, '_Enum__value'):
            return
        print(f'Initializing new {self.__class__}')
        # make sure the passed in value is a valid enumeration value
        if value not in self.__class__._enumerations.values():
            raise ValueError(f'{value} is not a valid {self.__class__.__name__}')
        # save the actual enumeration value
        for k, v in self.__class__._enumerations.items():
            if v == value:
                self.__key = k
                self.__value = v
                
    def __str__(self):
        return "%s.%s" % (self.__class__.__name__, self.__key)
    
    def __eq__(self, other):
        if type(self) != type(other):
            return False
        else:
            return (self.__key == other.__key and self.__value == other.__value)
    
class Color(Enum):
    RED: int = auto
    GREEN: int = auto
    BLUE: int = auto

class WeekendDay(Enum):
    SATURDAY: int = auto
    SUNDAY: int = auto
        
print(Color.RED == Color)               # False
print(Color.RED == Color.RED)           # True
print(Color.RED is Color.RED)           # True
print(Color.RED == Color['RED'])        # True
print(Color.RED == Color(1))            # True
print(Color.RED == Color(2))            # False
print(Color.RED == WeekendDay.SATURDAY) # False
print(Color.RED is Color.RED)           # True
print(Color.RED is Color['RED'])        # True
print(Color.RED is Color(1))            # True
for _ in range(100):
    c = Color(3)
    d = WeekendDay(2)

Constructing new type Enum
Constructing new type Color
Initializing new <enum 'Color'>
Initializing new <enum 'Color'>
Initializing new <enum 'Color'>
Constructing new type WeekendDay
Initializing new <enum 'WeekendDay'>
Initializing new <enum 'WeekendDay'>
False
True
True
True
True
False
False
True
True
True


In [13]:
# final version

class auto():
    pass

class EnumMeta(type):
    def __new__(metacls, cls, bases, classdict, **kwds):
        enumerations = {x: y for x, y in classdict.items() if not x.startswith('__')}
        # handle auto() and auto
        next_value = 1
        for k, v in enumerations.items():
            if type(v) is auto or v is auto:
                enumerations[k] = next_value
                next_value += 1
            else:
                next_value = v + 1
        enum = super().__new__(metacls, cls, bases, classdict, **kwds)
        enum._enumerations = enumerations
        reverse_enumerations = {y: x for x, y in enumerations.items()}
        # make "static" instances of each enumeration object
        enum._instances = {k: object.__new__(enum) for k, v in enumerations.items()}
        # initialize static instances
        for k, instance in enum._instances.items():
            instance.__init__(enumerations[k])
        # overwrite the new Enum's __new__() so that is returns the static instances
        enum.__new__ = lambda cls, value: enum._instances[reverse_enumerations[value]]
        return enum

    def __str__(cls):
        return f'<enum \'{cls.__name__}\'>'
    
    def __len__(cls):
        return len(cls._enumerations)
    
    def __iter__(cls):
        return (cls(value) for value in cls._enumerations.values())
        
    def __getitem__(cls, key):
        return cls(cls._enumerations[key])
    
    def __getattribute__(cls, key):
        if key.startswith('_'):
            return object.__getattribute__(cls, key)
        else:
            return cls(object.__getattribute__(cls, '_enumerations')[key])
        
    def __contains__(cls, other):
        if type(other) == cls:
            return True
        else:
            return False
        
class Enum(metaclass=EnumMeta):    
    def __init__(self, value):
        if hasattr(self, '_Enum__key') and hasattr(self, '_Enum__value'):
            return
        # make sure the passed in value is a valid enumeration value
        if value not in self.__class__._enumerations.values():
            raise ValueError(f'{value} is not a valid {self.__class__.__name__}')
        # save the actual enumeration value
        for k, v in self.__class__._enumerations.items():
            if v == value:
                self.__key = k
                self.__value = v
                
    def __str__(self):
        return "%s.%s" % (self.__class__.__name__, self.__key)
    
    def __eq__(self, other):
        if type(self) != type(other):
            return False
        else:
            return (self.__key == other.__key and self.__value == other.__value)